# 1-D Eigenvalue problems (Unit 2.2) 

We solve the generalized eigenvalue problem
$$A u=\lambda M u$$,

where A comes from $\int \nabla u \nabla v$, and $M$ from $\int uv$, on the space $H_0^1$

In [1]:
import netgen.gui
%gui tk
from ngsolve import *
import ngsolve.internal as ngint
ngint.viewoptions.drawedges = 1
from math import pi
import scipy.linalg
from scipy import random 
from random import random
from mesh_util import uniform_1d_mesh
mesh = Mesh(uniform_1d_mesh())


We setup a stiffness matrix A and a mass matrix M, and declare a preconditioner for A:

In [2]:
fes = H1(mesh, order=4, dirichlet='left|right')
u = fes.TrialFunction()
v = fes.TestFunction()

a = BilinearForm(fes)
a += SymbolicBFI(grad(u)*grad(v))
pre = Preconditioner(a, "direct")

m = BilinearForm(fes)
m += SymbolicBFI(u*v)

a.Assemble()
m.Assemble()

u = GridFunction(fes)

The inverse iteration is
$$u_{n+1}=A^{−1}Mu_n,$$

where the Rayleigh quotient
$$\rho_n=\frac{\langle Au_n,u_n \rangle}{\langle Mu_n,u_n\rangle}$$

converges to the smallest eigen-value, with rate of convergence $\lambda_1/\lambda_2$.

The preconditioned inverse iteration (PINVIT), see [Knyazef+Neymeyr], replaces $A^{−1}$
by an approximate inverse $C^{−1}$

$$\rho_n=\frac{\langle Au_n,u_n\rangle}{\langle Mu_n,u_n\rangle}$$

$$w_n=C^{−1}(Au_n−\rho Mu_n)$$
$$u_{n+1}=u_n+\alpha w_n$$

The optimal step-size $\alpha$ is found by minimizing the Rayleigh-quotient on a two-dimensional space:
$$u_{n+1}=\arg \min_{v\in \lbrace u_n,w_n\rbrace} \frac{\langle Av,v\rangle}{\langle Mv,v\rangle}$$

This minimization problem can be solved by a small eigenvalue problem
$$ay=\lambda my$$

with matrices
$$a = \left(
\begin{array}[cc]
\ \langle Au_n,u_n\rangle & \langle Au_n,w_n\rangle\\
\langle Aw_n,u_n\rangle & \langle Aw_n,w_n\rangle
\end{array}
\right) \hspace{3em} m = \left(
\begin{array}[cc]
\ \langle Mu_n,u_n\rangle & \langle Mu_n,w_n\rangle\\
\langle Mw_n,u_n\rangle & \langle Mw_n,w_n\rangle
\end{array}
\right) $$

Then, the new iterate is

$$u_{n+1}=y_1 u_n+y_2 w_n$$

where $y$ is the eigenvector corresponding to the smaller eigenvalue.
### Implementation in NGSolve

Create some help vectors:


In [3]:
r = u.vec.CreateVector()
w = u.vec.CreateVector()
Mu = u.vec.CreateVector()
Au = u.vec.CreateVector()
Mw = u.vec.CreateVector()
Aw = u.vec.CreateVector()


random initial condition, zero on the boundary

In [4]:
freedofs = fes.FreeDofs()    
for i in range(len(u.vec)):
    u.vec[i] = random() if freedofs[i] else 0   

In [5]:
for i in range(20):
    Au.data = a.mat * u.vec
    Mu.data = m.mat * u.vec
    auu = InnerProduct(Au, u.vec)
    muu = InnerProduct(Mu, u.vec)
    # Rayleigh quotiont
    lam = auu/muu
    print (lam / (pi**2))
    # residual
    r.data = Au - lam * Mu
    w.data = pre.mat * r.data
    w.data = 1/Norm(w) * w
    Aw.data = a.mat * w
    Mw.data = m.mat * w

    # setup and solve 2x2 small eigenvalue problem
    asmall = Matrix(2,2)
    asmall[0,0] = auu
    asmall[0,1] = asmall[1,0] = InnerProduct(Au, w)
    asmall[1,1] = InnerProduct(Aw, w)
    msmall = Matrix(2,2)
    msmall[0,0] = muu
    msmall[0,1] = msmall[1,0] = InnerProduct(Mu, w)
    msmall[1,1] = InnerProduct(Mw, w)
    # print ("asmall =", asmall, ", msmall = ", msmall)
    
    
    eval,evec = scipy.linalg.eigh(asmall, b=msmall)
    # print (eval / pi**2)
    # print (evec)
    u.vec.data = float(evec[0,0]) * u.vec + float(evec[1,0]) * w
    

8.16693633319564
1.0199017910447783
1.0002104803634897
1.0000029139032498
1.0000000524645536
1.0000000009609438
1.0000000000213254
1.0000000000040499
1.0000000000037315
1.000000000003726
1.000000000003726
1.0000000000037264
1.0000000000037268
1.0000000000037261
1.0000000000037266
1.0000000000037264
1.0000000000037266
1.0000000000037261
1.0000000000037264
1.000000000003726



### Simultaneous iteration for several eigenvalues

Declare GridFunction with multiple components to store several eigenvectors.


In [6]:
num = 5
u = GridFunction(fes, multidim=num)


Create list of help vectors:


In [7]:
r = u.vec.CreateVector()
Av = u.vec.CreateVector()
Mv = u.vec.CreateVector()

vecs = []
for i in range(2*num):
    vecs.append (u.vec.CreateVector())

freedofs = fes.FreeDofs()    
for v in u.vecs:
    for i in range(len(u.vec)):
        v[i] = random() if freedofs[i] else 0



Compute $num$ residuals, and solve small eigenvalue problem on $2num$-dimensional space

In [8]:
lams = num * [1]

asmall = Matrix(2*num, 2*num)
msmall = Matrix(2*num, 2*num)

for i in range(100):
    
    for j in range(num):
        vecs[j].data = u.vecs[j]
        r.data = a.mat * vecs[j] - lams[j] * m.mat * vecs[j]
        # r.data = 1/Norm(r) * r
        r *= 1/Norm(r)
        vecs[num+j].data = pre.mat * r

    for j in range(2*num):
        Av.data = a.mat * vecs[j]
        Mv.data = m.mat * vecs[j]
        for k in range(2*num):
            asmall[j,k] = InnerProduct(Av, vecs[k])
            msmall[j,k] = InnerProduct(Mv, vecs[k])

    ev,evec = scipy.linalg.eigh(a=asmall, b=msmall)
    lams[0:num] = ev[0:num]
    print (i, ":", [lam/pi**2 for lam in lams])
    
    for j in range(num):
        r[:] = 0.0
        for k in range(2*num):
            r.data += float(evec[k,j]) * vecs[k]
        u.vecs[j].data = r
    
Draw (u)

0 : [1.0019388379943166, 4.1108360115307319, 9.5965455500714771, 21.40876280137352, 35.602354410123098]
1 : [1.00000038737981, 4.0001693511283483, 9.0185453938025031, 16.391815754410647, 26.541041214687027]
2 : [1.0000000000647182, 4.0000002394055585, 9.0003641751997119, 16.022979566833538, 25.356015044603858]
3 : [1.0000000000037357, 4.0000000042453694, 9.0000068124981638, 16.001503838196946, 25.091356257101047]
4 : [1.0000000000037259, 4.0000000037860577, 9.000000336920154, 16.000110151910647, 25.024255642571493]
5 : [1.0000000000037264, 4.0000000037849341, 9.000000217650042, 16.000011581537507, 25.006503240135171]
6 : [1.0000000000037268, 4.0000000037848693, 9.0000002154348078, 16.000004341896883, 25.001770088844832]
7 : [1.0000000000037259, 4.0000000037849244, 9.0000002153932854, 16.000003799012422, 25.00050066550801]
8 : [1.0000000000037264, 4.0000000037848666, 9.0000002153924896, 16.000003757967484, 25.000160015464481]
9 : [1.0000000000037264, 4.0000000037848702, 9.00000021539243

The multidim-component select in the Visualization dialog box allows to switch between the components of the multidim-GridFunction.